# Real-Time Crowd Monitoring with Sudden Speed Change Detection using YOLOv8

In [ ]:
# Install PyTorch (for YOLOv8 and deep learning support)
!pip install torch torchvision torchaudio

# Install Ultralytics (for YOLOv8)
!pip install ultralytics

# Install OpenCV (for video processing)
!pip install opencv-python-headless

# Install Supervision (used for annotations)
!pip install supervision==0.2.0

# Install MongoDB client (for saving detection data, optional)
!pip install pymongo

# Install dotenv (for loading environment variables)
!pip install python-dotenv

# Install Flask 
!pip install flask


In [1]:
import torch
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_03:03:05_Pacific_Daylight_Time_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
torch:  2.3 ; cuda:  cpu


In [2]:
from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.2.64  Python-3.11.7 torch-2.3.1+cpu CPU (13th Gen Intel Core(TM) i7-13700H)
Setup complete  (20 CPUs, 15.6 GB RAM, 316.8/440.1 GB disk)


In [3]:
!pip install supervision==0.2.0

from IPython import display
display.clear_output()

import supervision as sv
print("supervision", sv.__version__)

supervision 0.2.0


In [1]:
from flask import Flask, Response
import cv2
import numpy as np
from ultralytics import YOLO
from pymongo import MongoClient
from datetime import datetime, date
import time
from dotenv import load_dotenv
import os

# Load YOLO model
model = YOLO('yolov8n.pt')  # or use a different YOLO version

# Connect to the RTSP stream
cap = cv2.VideoCapture(0)

# MongoDB connection  (optional) (commented out for now)
# Uncomment the lines below if you are using MongoDB to store the data
'''
client = MongoClient('')
db = client["CrowdTracking"]
collection = db["Crowd"]
'''

# Variables for frame_id and date format
frame_id = 0
current_date = date.today()
update_interval = 1  # Update interval in seconds
last_update_time = 0

# To track positions and speeds
prev_positions = []  # Positions in the previous frame
speed_threshold = 45 # Speed threshold to trigger warning (adjust as needed)
tracking_distance_threshold = 50  # Maximum allowed distance to associate a person in consecutive frames

# Function to associate detected persons between frames based on proximity
def associate_persons(prev_positions, current_positions):
    matched_positions = []
    for curr_pos in current_positions:
        # Find the closest previous position for each current position
        min_distance = float('inf')
        matched_prev_pos = None
        for prev_pos in prev_positions:
            distance = np.linalg.norm(np.array(curr_pos) - np.array(prev_pos))
            if distance < min_distance and distance < tracking_distance_threshold:
                min_distance = distance
                matched_prev_pos = prev_pos
        if matched_prev_pos is not None:
            matched_positions.append((matched_prev_pos, curr_pos))
    return matched_positions

while True:
    current_time = time.time()
    ret, frame = cap.read()
    if not ret:
        print('Error reading the frame')
        break

    # Perform YOLO detection
    results = model(frame)

    # Current positions
    current_positions = []
    for result in results:
        boxes = result.boxes.cpu().numpy()
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            cls = int(box.cls[0])

            if cls == 0:  # Assuming class 0 is person
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, f'Person: {conf:.2f}', (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
                current_positions.append(((x1 + x2) // 2, (y1 + y2) // 2))  # Center of the box

    # Associate persons across frames and calculate speeds
    sudden_speed_detected = False
    if prev_positions:
        matched_positions = associate_persons(prev_positions, current_positions)
        for prev_pos, curr_pos in matched_positions:
            speed = np.linalg.norm(np.array(curr_pos) - np.array(prev_pos))
            if speed > speed_threshold:
                sudden_speed_detected = True
                break

    prev_positions = current_positions  # Update previous positions for the next frame

    # Display warning if sudden speed detected
    if sudden_speed_detected:
        cv2.putText(frame, 'Warning: Sudden Speed Change!', (50, 100),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)  # Display warning

    # This update allows saving the number of persons detected to MongoDB
    # for every update_interval seconds
    if current_time - last_update_time < update_interval:
        now = datetime.now()
        # Save the number of persons detected to MongoDB
        data = {
            "frame_id": frame_id,
            "timestamp": now.strftime("%d/%m/%Y %H:%M:%S"),
            "total_persons": len(boxes)
        }
        # collection.insert_one(data)
        last_update_time = current_time

    # Display the number of persons detected on the frame
    cv2.rectangle(frame, (10, 10), (310, 60), (255, 255, 255), -1)
    cv2.putText(frame, f'Total Persons: {len(current_positions)}', (20, 40),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    frame_id += 1

    # Display the frame
    cv2.imshow('Crowd Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 128.7ms
Speed: 5.0ms preprocess, 128.7ms inference, 16.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 dog, 76.8ms
Speed: 0.0ms preprocess, 76.8ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 80.6ms
Speed: 0.0ms preprocess, 80.6ms inference, 0.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 dog, 83.0ms
Speed: 3.8ms preprocess, 83.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 75.3ms
Speed: 2.0ms preprocess, 75.3ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 1 dog, 67.3ms
Speed: 2.0ms preprocess, 67.3ms inference, 1.8ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 74.5ms
Speed: 0.0ms preprocess, 74.5ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 53.3ms
Speed: 1.3ms preprocess, 53.3ms inference, 0.0ms postprocess per 